In [20]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [21]:
def chat_bot():
  model_name = "Qwen/Qwen2-1.5B-Instruct"
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModelForCausalLM.from_pretrained(
      model_name,
      torch_dtype=torch.float16, # Using float16 for small models is fine
      device_map="auto",
  )

  print("\nChatBot is ready! Type 'quit' to stop.")
  print("="*30)

  # The chat loop
  for step in range(10):
      user_input = input("You: ")
      if user_input.lower() == "quit":
          break

      # Format the input correctly for this instruction-tuned model
      messages = [
          {"role": "user", "content": user_input}
      ]
      text = tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
      )

      model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

      # Generate a response
      generated_ids = model.generate(
          model_inputs.input_ids,
          max_new_tokens=512
      )

      generated_ids = [
          output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
      ]

      # Decode the response
      response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

      print(f"Bot: {response}")

In [14]:
def sentiment_analysis():
    print("\n--- Sentiment Analysis ---")
    sentiment = pipeline("sentiment-analysis")
    text = input('''Enter text to analyze sentiment: ''')
    result = sentiment(text)[0]
    print(f"Label: {result['label']} | Score: {result['score']:.2f}")

In [15]:
def text_generation():
    print("\n--- Text Generation ---")
    generator = pipeline("text-generation", model="gpt2")
    prompt = input('''Enter a prompt for text generation: ''')
    result = generator(prompt, max_length=100, num_return_sequences=1)
    print("Generated text:\n", result[0]['generated_text'])

In [16]:
def ner():
    print("\n--- Named Entity Recognition ---")
    ner_model = pipeline("ner", grouped_entities=True)
    text = input('''Enter text for NER: ''')
    result = ner_model(text)
    for entity in result:
        print(f"{entity['word']} ({entity['entity_group']}) - score: {entity['score']:.2f}")


In [18]:
def question_answering():
    print("\n--- Question Answering ---")
    qa = pipeline("question-answering")
    question = input('''Enter your question: ''')
    context = input('''Enter your question: ''')
    result = qa(question=question , context = context)
    print(f"Answer = {result['answer']}")

In [24]:
chat_bot()


ChatBot is ready! Type 'quit' to stop.
You: quit


In [19]:
ner()

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.



--- Named Entity Recognition ---


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


Enter text for NER: hello you are amazing in pakistan
##akistan (LOC) - score: 0.96


In [28]:
!pip install -q gradio


In [29]:
import gradio as gr
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# Load the Qwen model
model_name = "Qwen/Qwen2-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Other pipelines
sentiment = pipeline("sentiment-analysis")
generator = pipeline("text-generation", model="gpt2")
ner_model = pipeline("ner", grouped_entities=True)
qa = pipeline("question-answering")

# ChatBot Response Function
def chat_with_bot(user_input, history=[]):
    messages = [{"role": "user", "content": user_input}]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=512)
    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

# Other feature functions
def do_sentiment(text):
    result = sentiment(text)[0]
    return f"Label: {result['label']}, Score: {result['score']:.2f}"

def generate_text(prompt):
    result = generator(prompt, max_length=100, num_return_sequences=1)
    return result[0]['generated_text']

def named_entities(text):
    result = ner_model(text)
    return "\n".join(f"{e['word']} ({e['entity_group']}) - {e['score']:.2f}" for e in result)

def answer_question(question, context):
    result = qa(question=question, context=context)
    return result["answer"]

# UI Tabs
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("🌟 AI NLP Assistant – Powered by Transformers")

    with gr.Tab("💬 ChatBot"):
        chatbot = gr.ChatInterface(fn=chat_with_bot)

    with gr.Tab("🧠 Sentiment Analysis"):
        text = gr.Textbox(label="Enter text")
        sent_btn = gr.Button("Analyze")
        sent_output = gr.Textbox(label="Result")
        sent_btn.click(do_sentiment, inputs=text, outputs=sent_output)

    with gr.Tab("✍️ Text Generation"):
        prompt = gr.Textbox(label="Enter prompt")
        gen_btn = gr.Button("Generate Text")
        gen_output = gr.Textbox(label="Generated")
        gen_btn.click(generate_text, inputs=prompt, outputs=gen_output)

    with gr.Tab("🔍 Named Entity Recognition"):
        ner_input = gr.Textbox(label="Enter text")
        ner_btn = gr.Button("Find Entities")
        ner_output = gr.Textbox(label="Entities Found")
        ner_btn.click(named_entities, inputs=ner_input, outputs=ner_output)

    with gr.Tab("❓ Question Answering"):
        q = gr.Textbox(label="Question")
        ctx = gr.Textbox(label="Context")
        ans_btn = gr.Button("Get Answer")
        ans_output = gr.Textbox(label="Answer")
        ans_btn.click(answer_question, inputs=[q, ctx], outputs=ans_output)

demo.launch()


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
Device set to use cpu
No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a Ber

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://88fb0cbec85927a86a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
